# ComfyUI (GGUF) ? LTX-2 (Colab)

This notebook:
- installs ComfyUI + ComfyUI-Manager + ComfyUI-GGUF
- downloads LTX-2 GGUF + required text/connector/VAE components
- launches ComfyUI via Cloudflare Tunnel

Quick start:
1. Run all cells top-to-bottom.
2. Provide HF token when prompted.
3. Import `workflow-ltx2-dev.json` (or distilled workflow) in ComfyUI.


## 1) Installation
Install ComfyUI, ComfyUI-Manager, ComfyUI-GGUF, and swap for better Colab stability.


In [ ]:
# @title 1) Install ComfyUI + Manager + ComfyUI-GGUF (with swap)
import os

# Small protection against RAM-related crashes
if not os.path.exists('/swapfile'):
    print('Creating swap (8GB)...')
    !sudo fallocate -l 8G /swapfile
    !sudo chmod 600 /swapfile
    !sudo mkswap /swapfile
    !sudo swapon /swapfile
    print('Swap enabled.')

# Useful packages
!apt-get -y update -qq
!apt-get -y install -qq aria2 ffmpeg

# More predictable CUDA allocator (sometimes helps reduce fragmentation)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# ComfyUI
if not os.path.exists('/content/ComfyUI'):
    print('Cloning ComfyUI...')
    !git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI

%cd /content/ComfyUI

print('Installing python requirements...')
!pip install -U pip
!pip install -r requirements.txt

# Attention acceleration pack (T4-safe)
ENABLE_ACCEL_PACK = True  # Set False to skip optional attention accelerators.

if ENABLE_ACCEL_PACK:
    import sys
    import subprocess

    def pip_try(spec):
        print(f"Installing optional accelerator: {spec}")
        return subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', spec], check=False).returncode == 0

    import torch
    gpu_name = 'CPU'
    cc_major, cc_minor = (0, 0)
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        cc_major, cc_minor = torch.cuda.get_device_capability(0)

    print(f"GPU detected: {gpu_name} (sm{cc_major}{cc_minor})")

    x_ok = pip_try('xformers>=0.0.28.post3')
    print('xformers:', 'OK' if x_ok else 'FAILED (continuing)')

    # FlashAttention-3 is Hopper-only; FlashAttention-2 and SageAttention target Ampere+.
    if cc_major >= 8:
        fa_ok = pip_try('flash-attn>=2.7.0.post2')
        sage_ok = pip_try('sageattention>=2.1.1')
        print('flash-attn:', 'OK' if fa_ok else 'FAILED (continuing)')
        print('sageattention:', 'OK' if sage_ok else 'FAILED (continuing)')
    else:
        print('Skipping flash-attn and sageattention on this GPU: T4/Turing (sm75) uses xformers path.')
else:
    print('Attention acceleration pack disabled by user (ENABLE_ACCEL_PACK=False).')


# ComfyUI-Manager
if not os.path.exists('custom_nodes/ComfyUI-Manager'):
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git custom_nodes/ComfyUI-Manager

# ComfyUI-GGUF
if not os.path.exists('custom_nodes/ComfyUI-GGUF'):
    print('Installing ComfyUI-GGUF...')
    !git clone https://github.com/city96/ComfyUI-GGUF.git custom_nodes/ComfyUI-GGUF
    !pip install -r custom_nodes/ComfyUI-GGUF/requirements.txt

# ComfyUI-KJNodes
if not os.path.exists('custom_nodes/comfyui-kjnodes'):
    print('Installing ComfyUI-KJNodes...')
    !git clone https://github.com/kijai/ComfyUI-KJNodes.git custom_nodes/comfyui-kjnodes
if os.path.exists('custom_nodes/comfyui-kjnodes/requirements.txt'):
    !pip install -r custom_nodes/comfyui-kjnodes/requirements.txt



# Hugging Face authentication (interactive prompt)
!pip install -q huggingface_hub
from getpass import getpass
from huggingface_hub import login

hf_token = getpass('Enter Hugging Face token: ').strip()
if not hf_token:
    raise RuntimeError('Hugging Face token is required for this notebook run.')

login(token=hf_token, add_to_git_credential=False)
os.environ['HUGGINGFACE_TOKEN'] = hf_token
os.environ['HF_TOKEN'] = hf_token
print('HF auth: OK')

# Civitai authentication (for LoRA downloads)
civitai_token = getpass('Enter Civitai API token (optional, press Enter to skip): ').strip()
if civitai_token:
    os.environ['CIVITAI_API_TOKEN'] = civitai_token
    print('Civitai auth: OK')
else:
    print('Civitai token not set. Civitai downloads may fail (403).')

print('Done.')


## 2) Quant Settings
Choose GGUF quant defaults and review rough file-size/VRAM estimates.


In [ ]:
# @title 2) Settings: GGUF quant selection + size/VRAM table
import math

# ---- Auto quant by VRAM ----
AUTO_QUANT_BY_VRAM = True
AUTO_QUANT_VRAM_FRACTION = 0.9
AUTO_VRAM_FALLBACK_GB = 14.0


# ---- Quant selection (editable) ----
MAIN_MODEL = 'dev'      # options: 'dev', 'distilled'
LTX_QUANT = 'q2_k_s'    # dev options: q2_k_s, q2_k, iq4_xs, iq4_nl | distilled: q2_k

# Text encoder for LTX2 GGUF setup (currently only this quant is published)
TEXT_ENCODER_GGUF = 'gemma3-12b-it-q2_k.gguf'

# If VRAM is limited, keep this enabled to offload text encoder side to CPU/RAM
OFFLOAD_TEXT_ENCODER = True

# Required component for workflow-ltx2-dev/distilled (audio branch)
DOWNLOAD_AUDIO_VAE = True

# Required LoRAs used in the default LTX workflows
DOWNLOAD_LTX_WORKFLOW_LORAS = True

# Optional LTX-2 spatial upscaler x2 (very heavy, disabled by default for T4 free tier)
DOWNLOAD_SPATIAL_UPSCALER_X2 = False

# ---- File-size tables (GB), checked from repo headers ----
LTX_DEV_SIZES_GB = {
    'q2_k_s': 7.07,
    'q2_k': 8.10,
    'iq4_xs': 10.57,
    'iq4_nl': 11.15,
}

LTX_DISTILLED_SIZES_GB = {
    'q2_k': 7.06,
}

TEXT_ENCODER_SIZES_GB = {
    'q2_k': 4.77,
}

CONNECTOR_GB = 1.43
VIDEO_VAE_GB = 1.25
AUDIO_VAE_GB = 0.22
TAE_LTX2_GB = 0.03
LTX_WORKFLOW_LORAS_GB = 0.55
SPATIAL_UPSCALER_GB = 10.86




def detect_total_vram_gb(default=AUTO_VRAM_FALLBACK_GB):
    try:
        import torch
        if torch.cuda.is_available():
            return torch.cuda.get_device_properties(0).total_memory / (1024**3)
    except Exception as e:
        print(f"VRAM detection failed ({e}); using fallback {default:.2f} GB")
    return default


def pick_best_quant(size_dict, budget_gb, overhead=1.10):
    items = sorted(size_dict.items(), key=lambda kv: kv[1])
    fitting = [(k, v) for k, v in items if (v * overhead) <= budget_gb]
    if fitting:
        return fitting[-1][0]
    return items[0][0]


def est_vram_gb(size_gb, overhead=1.10):
    return size_gb * overhead


def print_table(title, d):
    print('\n' + title)
    print('-' * len(title))
    for k, v in d.items():
        print(f"{k:10s}  file~{v:5.2f} GB   VRAM~{est_vram_gb(v):5.2f} GB")


if MAIN_MODEL == 'distilled' and LTX_QUANT != 'q2_k':
    print('Warning: distilled currently supports only q2_k in this notebook. Forcing q2_k.')
    LTX_QUANT = 'q2_k'

# ---- Auto apply quant choice based on detected VRAM ----
TOTAL_VRAM_GB = detect_total_vram_gb()
AUTO_BUDGET_GB = TOTAL_VRAM_GB * AUTO_QUANT_VRAM_FRACTION
print(f"\nGPU VRAM detected: ~{TOTAL_VRAM_GB:.2f} GB | Auto budget (75%): ~{AUTO_BUDGET_GB:.2f} GB")

if AUTO_QUANT_BY_VRAM:
    model_tbl = LTX_DEV_SIZES_GB if MAIN_MODEL == 'dev' else LTX_DISTILLED_SIZES_GB
    fixed_side = est_vram_gb(VIDEO_VAE_GB) + est_vram_gb(TAE_LTX2_GB)
    if DOWNLOAD_AUDIO_VAE:
        fixed_side += est_vram_gb(AUDIO_VAE_GB)
    if DOWNLOAD_LTX_WORKFLOW_LORAS:
        fixed_side += est_vram_gb(LTX_WORKFLOW_LORAS_GB)
    if DOWNLOAD_SPATIAL_UPSCALER_X2:
        fixed_side += est_vram_gb(SPATIAL_UPSCALER_GB)
    if not OFFLOAD_TEXT_ENCODER:
        fixed_side += est_vram_gb(TEXT_ENCODER_SIZES_GB['q2_k']) + est_vram_gb(CONNECTOR_GB)

    budget_main = max(AUTO_BUDGET_GB - fixed_side, 0.01)
    LTX_QUANT = pick_best_quant(model_tbl, budget_main)
    print(f"Auto quant active: LTX_QUANT={LTX_QUANT} (MAIN_MODEL={MAIN_MODEL})")

print_table('LTX2 dev GGUF quants', LTX_DEV_SIZES_GB)
print_table('LTX2 distilled GGUF quants', LTX_DISTILLED_SIZES_GB)
print_table('Gemma3-12B encoder GGUF quants (currently available)', TEXT_ENCODER_SIZES_GB)

if MAIN_MODEL == 'dev':
    main_size = LTX_DEV_SIZES_GB[LTX_QUANT]
else:
    main_size = LTX_DISTILLED_SIZES_GB[LTX_QUANT]

print('\nSelected:')
print('  MAIN_MODEL           =', MAIN_MODEL)
print('  LTX_QUANT            =', LTX_QUANT)
print('  OFFLOAD_TEXT_ENCODER =', OFFLOAD_TEXT_ENCODER)
print('  DOWNLOAD_AUDIO_VAE   =', DOWNLOAD_AUDIO_VAE)
print('  DOWNLOAD_LTX_WORKFLOW_LORAS =', DOWNLOAD_LTX_WORKFLOW_LORAS)
print('  DOWNLOAD_SPATIAL_UPSCALER_X2 =', DOWNLOAD_SPATIAL_UPSCALER_X2)

# Rough totals
always_gpu = est_vram_gb(main_size) + est_vram_gb(VIDEO_VAE_GB) + est_vram_gb(TAE_LTX2_GB)
if DOWNLOAD_AUDIO_VAE:
    always_gpu += est_vram_gb(AUDIO_VAE_GB)
if DOWNLOAD_LTX_WORKFLOW_LORAS:
    always_gpu += est_vram_gb(LTX_WORKFLOW_LORAS_GB)
if DOWNLOAD_SPATIAL_UPSCALER_X2:
    always_gpu += est_vram_gb(SPATIAL_UPSCALER_GB)

text_side = est_vram_gb(TEXT_ENCODER_SIZES_GB['q2_k']) + est_vram_gb(CONNECTOR_GB)

if OFFLOAD_TEXT_ENCODER:
    total_gpu = always_gpu
else:
    total_gpu = always_gpu + text_side

print(f"\nRough GPU VRAM for weights: ~{total_gpu:.2f} GB")
print('Note: real peak VRAM depends on frames/resolution/scheduler/latents and can be higher.')


## 3) Model Download
Download the required model files (and optional components) into ComfyUI folders.


In [ ]:
# @title 3) Download models (LTX2 GGUF + text encoder + connector + VAE + workflow LoRAs)
import os

COMFY = '/content/ComfyUI'

UNET_DIR = f'{COMFY}/models/unet'
DIFF_DIR = f'{COMFY}/models/diffusion_models'
TE_DIR   = f'{COMFY}/models/text_encoders'
CLIP_DIR = f'{COMFY}/models/clip'
VAE_DIR  = f'{COMFY}/models/vae'
VAE_APPROX_DIR = f'{COMFY}/models/vae_approx'
CKPT_DIR = f'{COMFY}/models/checkpoints'
UPSCALE_DIR = f'{COMFY}/models/latent_upscale_models'
LORA_DIR = f'{COMFY}/models/loras'
WORKFLOW_DIR = f'{COMFY}/user/default/workflows'

for d in [UNET_DIR, DIFF_DIR, TE_DIR, CLIP_DIR, VAE_DIR, VAE_APPROX_DIR, CKPT_DIR, UPSCALE_DIR, LORA_DIR, WORKFLOW_DIR]:
    os.makedirs(d, exist_ok=True)


def dl(url, outdir, fname):
    outpath = os.path.join(outdir, fname)
    if os.path.exists(outpath):
        print('Already exists:', outpath)
        return
    print('Downloading:', fname)

    hf_token = os.environ.get('HUGGINGFACE_TOKEN') or os.environ.get('HF_TOKEN')
    if hf_token and 'huggingface.co' in url:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --header="Authorization: Bearer {hf_token}" "{url}" -d "{outdir}" -o "{fname}"
    else:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{outdir}" -o "{fname}"


# Main LTX2 GGUF
if MAIN_MODEL == 'dev':
    main_fname = f"ltx2-19b-dev-{LTX_QUANT}.gguf"
else:
    main_fname = f"ltx2-19b-distilled-{LTX_QUANT}.gguf"

main_url = f"https://huggingface.co/gguf-org/ltx2-gguf/resolve/main/{main_fname}"
dl(main_url, DIFF_DIR, main_fname)

main_link = f"{UNET_DIR}/{main_fname}"
if not os.path.exists(main_link):
    !ln -s "{DIFF_DIR}/{main_fname}" "{main_link}"

# Text side
text_fname = 'gemma3-12b-it-q2_k.gguf'
text_url = f"https://huggingface.co/gguf-org/ltx2-gguf/resolve/main/{text_fname}"
dl(text_url, TE_DIR, text_fname)

text_clip_link = f"{CLIP_DIR}/{text_fname}"
if not os.path.exists(text_clip_link):
    !ln -s "{TE_DIR}/{text_fname}" "{text_clip_link}"

connector_fname = 'ltx2-19b-embeddings_connector_dev_fp8_e4m3fn.safetensors'
connector_url = f"https://huggingface.co/gguf-org/ltx2-gguf/resolve/main/{connector_fname}"
dl(connector_url, TE_DIR, connector_fname)

# VAE
video_vae_fname = 'ltx2_video_vae_fp8_e4m3fn.safetensors'
video_vae_url = f"https://huggingface.co/gguf-org/ltx2-gguf/resolve/main/{video_vae_fname}"
dl(video_vae_url, VAE_DIR, video_vae_fname)

tae_vae_fname = 'taeltx_2.safetensors'
tae_vae_url = "https://huggingface.co/Kijai/LTXV2_comfy/resolve/main/VAE/taeltx_2.safetensors"
dl(tae_vae_url, VAE_APPROX_DIR, tae_vae_fname)

if DOWNLOAD_AUDIO_VAE:
    audio_vae_fname = 'ltx2_audio_vae_fp8_e4m3fn.safetensors'
    audio_vae_url = f"https://huggingface.co/gguf-org/ltx2-gguf/resolve/main/{audio_vae_fname}"
    dl(audio_vae_url, VAE_DIR, audio_vae_fname)

if DOWNLOAD_LTX_WORKFLOW_LORAS:
    lora_distilled_fname = 'ltx-2-19b-distilled-lora-384.safetensors'
    lora_distilled_url = 'https://huggingface.co/Lightricks/LTX-2/resolve/main/ltx-2-19b-distilled-lora-384.safetensors'
    dl(lora_distilled_url, LORA_DIR, lora_distilled_fname)

    lora_camera_fname = 'ltx-2-19b-lora-camera-control-dolly-left.safetensors'
    lora_camera_url = 'https://huggingface.co/Lightricks/LTX-2-19b-LoRA-Camera-Control-Dolly-Left/resolve/main/ltx-2-19b-lora-camera-control-dolly-left.safetensors'
    dl(lora_camera_url, LORA_DIR, lora_camera_fname)

if DOWNLOAD_SPATIAL_UPSCALER_X2:
    upscaler_fname = 'ltx-2-spatial-upscaler-x2-1.0.safetensors'
    upscaler_url = 'https://huggingface.co/Lightricks/LTX-2/resolve/main/ltx-2-spatial-upscaler-x2-1.0.safetensors'
    dl(upscaler_url, UPSCALE_DIR, upscaler_fname)

# Optional: workflow JSON from same repo
workflow_name = 'workflow-ltx2-dev.json' if MAIN_MODEL == 'dev' else 'workflow-ltx2-distilled.json'
workflow_url = f"https://huggingface.co/gguf-org/ltx2-gguf/resolve/main/{workflow_name}"
dl(workflow_url, WORKFLOW_DIR, workflow_name)

print('Done downloading models.')
print('Main model :', main_fname)
print('Text encoder:', text_fname)
print('Connector   :', connector_fname)
print('Video VAE   :', video_vae_fname)
print('TAE (LTX2)  :', tae_vae_fname)
if DOWNLOAD_AUDIO_VAE:
    print('Audio VAE   :', audio_vae_fname)
if DOWNLOAD_LTX_WORKFLOW_LORAS:
    print('LoRAs       :', lora_distilled_fname, ',', lora_camera_fname)
if DOWNLOAD_SPATIAL_UPSCALER_X2:
    print('Spatial Upscaler x2:', upscaler_fname)
print('Workflow    :', workflow_name)


# ---- Lenovo UltraReal LoRA pack (Civitai) ----
DOWNLOAD_LENOVO_ULTRAREAL_LORAS = True
CIVITAI_API_TOKEN = os.environ.get('CIVITAI_API_TOKEN', '').strip()  # optional, helps if Civitai returns 403.
CURRENT_BASE_MODELS = []

if DOWNLOAD_LENOVO_ULTRAREAL_LORAS:
    import json
    import subprocess
    import urllib.request
    import urllib.parse

    LORA_DIR = f'{COMFY}/models/loras'
    os.makedirs(LORA_DIR, exist_ok=True)

    def add_civitai_token(url, token):
        if not token:
            return url
        parts = urllib.parse.urlsplit(url)
        query = urllib.parse.parse_qsl(parts.query, keep_blank_values=True)
        if not any(k == 'token' for k, _ in query):
            query.append(('token', token))
        return urllib.parse.urlunsplit((parts.scheme, parts.netloc, parts.path, urllib.parse.urlencode(query), parts.fragment))

    def dl_civitai(url, outdir, fname, token=''):
        outpath = os.path.join(outdir, fname)
        if os.path.exists(outpath):
            print('Already exists:', outpath)
            return True

        final_url = add_civitai_token(url, token)
        print('Downloading LoRA:', fname)
        cmd = ['aria2c', '--console-log-level=error', '-c', '-x', '8', '-s', '8', '-k', '1M', final_url, '-d', outdir, '-o', fname]
        rc = subprocess.run(cmd, check=False).returncode
        if rc != 0 and not token:
            print('  Download failed without token. Set CIVITAI_API_TOKEN env var and retry Cell 3.')
        return rc == 0

    try:
        req = urllib.request.Request(
            'https://civitai.com/api/v1/models/1662740',
            headers={'User-Agent': 'Mozilla/5.0'}
        )
        with urllib.request.urlopen(req, timeout=30) as resp:
            civitai_model = json.loads(resp.read().decode('utf-8'))

        versions = civitai_model.get('modelVersions', [])
        print('\nLenovo UltraReal LoRA versions found:', len(versions))
        if not CURRENT_BASE_MODELS:
            print('Current notebook base-model tags: (none matched explicitly)')
        else:
            print('Current notebook base-model tags:', ', '.join(CURRENT_BASE_MODELS))

        for mv in versions:
            base = mv.get('baseModel', 'Unknown')
            match = base in CURRENT_BASE_MODELS
            mark = 'MATCH' if match else 'OTHER'
            files = [f for f in mv.get('files', []) if f.get('type') == 'Model']
            print(f"- [{mark}] {base} :: {mv.get('name', 'Unnamed version')} ({len(files)} file(s))")

        ok_count = 0
        total_count = 0
        matched_versions = [mv for mv in versions if mv.get('baseModel', 'Unknown') in CURRENT_BASE_MODELS]

        if not matched_versions:
            print('No MATCH versions found for this notebook base-model tag set. Skipping LoRA download.')

        for mv in matched_versions:
            for fobj in mv.get('files', []):
                if fobj.get('type') != 'Model':
                    continue
                total_count += 1
                raw_name = (fobj.get('name') or '').strip()
                base_tag = ''.join(ch if ch.isalnum() else '_' for ch in str(mv.get('baseModel', 'unknown'))).strip('_').lower() or 'unknown'
                fname = f"lenovo_ultrareal_{base_tag}_v{mv.get('id', 'x')}_f{fobj.get('id', 'x')}.safetensors"
                if raw_name and raw_name != fname:
                    print(f"  Civitai file name: {raw_name} -> saved as {fname}")
                if dl_civitai(fobj.get('downloadUrl', ''), LORA_DIR, fname, CIVITAI_API_TOKEN):
                    ok_count += 1

        print(f"\nLenovo UltraReal LoRA download summary (MATCH only): {ok_count}/{total_count} files ready in {LORA_DIR}")

    except Exception as e:
        print('Failed to fetch Lenovo UltraReal LoRA metadata from Civitai:', e)
        print('Tip: this may require CIVITAI_API_TOKEN or temporary Civitai availability.')


## 4) Download Check
Verify downloaded file sizes and calculate a rough VRAM estimate from real files.


In [ ]:
# @title 4) Verify downloads: actual file sizes and VRAM estimate
import os


def size_gb(path):
    return os.path.getsize(path) / (1024**3)


def est_vram_from_file(path, overhead=1.10):
    return size_gb(path) * overhead


if MAIN_MODEL == 'dev':
    main_fname = f"ltx2-19b-dev-{LTX_QUANT}.gguf"
else:
    main_fname = f"ltx2-19b-distilled-{LTX_QUANT}.gguf"

paths = [
    ('/content/ComfyUI/models/diffusion_models', main_fname),
    ('/content/ComfyUI/models/text_encoders', 'gemma3-12b-it-q2_k.gguf'),
    ('/content/ComfyUI/models/text_encoders', 'ltx2-19b-embeddings_connector_dev_fp8_e4m3fn.safetensors'),
    ('/content/ComfyUI/models/vae', 'ltx2_video_vae_fp8_e4m3fn.safetensors'),
    ('/content/ComfyUI/models/vae_approx', 'taeltx_2.safetensors'),
]

if DOWNLOAD_AUDIO_VAE:
    paths.append(('/content/ComfyUI/models/vae', 'ltx2_audio_vae_fp8_e4m3fn.safetensors'))
if DOWNLOAD_LTX_WORKFLOW_LORAS:
    paths.append(('/content/ComfyUI/models/loras', 'ltx-2-19b-distilled-lora-384.safetensors'))
    paths.append(('/content/ComfyUI/models/loras', 'ltx-2-19b-lora-camera-control-dolly-left.safetensors'))
if DOWNLOAD_SPATIAL_UPSCALER_X2:
    paths.append(('/content/ComfyUI/models/latent_upscale_models', 'ltx-2-spatial-upscaler-x2-1.0.safetensors'))

print('Files:')
total_vram = 0.0
for d, f in paths:
    p = os.path.join(d, f)
    if not os.path.exists(p):
        print('MISSING:', p)
        continue
    s = size_gb(p)
    v = est_vram_from_file(p)
    total_vram += v
    print(f"- {p}\n  size~{s:.2f} GB  -> VRAM(weights)~{v:.2f} GB")

print(f"\nSum VRAM(weights) if all kept on GPU at once: ~{total_vram:.2f} GB")
print('Reminder: offloading text-side components can significantly reduce active GPU usage.')


## 5) Launch ComfyUI
Start ComfyUI and use the Cloudflare Tunnel URL shown in the output.


In [ ]:
# @title 5) Launch ComfyUI (Cloudflare Tunnel) - stable mode
import subprocess, threading, time, socket, os, re, sys, urllib.request

%cd /content/ComfyUI

# Install cloudflared (if not installed yet)
if not os.path.exists("cloudflared-linux-amd64.deb"):
    !wget -q -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb


def wait_port(host="127.0.0.1", port=8188, timeout=240):
    t0 = time.time()
    while time.time() - t0 < timeout:
        try:
            with socket.create_connection((host, port), timeout=1):
                return True
        except OSError:
            time.sleep(0.5)
    return False


def verify_tunnel_url(url, timeout=10):
    base = url.rstrip('/')
    candidates = [
        base + '/',
        base + '/api/system_stats',
        base + '/object_info',
    ]
    for c in candidates:
        try:
            req = urllib.request.Request(c, headers={'User-Agent': 'Mozilla/5.0'}, method='GET')
            with urllib.request.urlopen(req, timeout=timeout) as r:
                code = r.getcode()
                if 200 <= code < 500:
                    return True
        except Exception:
            pass
    return False


def wait_reachable_stable(url, proc, warmup_timeout=180, stable_successes=3, probe_interval=2.5):
    t0 = time.time()
    ok_streak = 0
    while time.time() - t0 < warmup_timeout:
        if proc.poll() is not None:
            return False
        if verify_tunnel_url(url, timeout=10):
            ok_streak += 1
            if ok_streak >= stable_successes:
                return True
        else:
            ok_streak = 0
        time.sleep(probe_interval)
    return False


def start_cloudflare_tunnel_once(port=8188, protocol='http2', read_timeout=150, warmup_timeout=180):
    print(f"\nStarting Cloudflare Quick Tunnel (protocol={protocol})...\n")
    sys.stdout.flush()

    cmd = [
        'cloudflared', 'tunnel',
        '--no-autoupdate',
        '--url', f'http://127.0.0.1:{port}',
        '--protocol', protocol,
        '--loglevel', 'info',
    ]

    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
    )

    tunnel_patterns = [
        re.compile(r'https://[a-z0-9-]+\.trycloudflare\.com', re.I),
        re.compile(r'https://[a-z0-9-]+\.cfargotunnel\.com', re.I),
    ]
    ignore_patterns = [
        re.compile(r'https://www\.cloudflare\.com/website-terms/?', re.I),
    ]

    t0 = time.time()
    url = None

    while time.time() - t0 < read_timeout:
        line = proc.stdout.readline()
        if not line:
            if proc.poll() is not None:
                break
            time.sleep(0.1)
            continue

        s = line.strip()

        if any(ip.search(s) for ip in ignore_patterns):
            continue

        for pat in tunnel_patterns:
            m = pat.search(s)
            if m:
                url = m.group(0)
                break
        if url:
            break

    if not url:
        print('Failed to parse tunnel URL.')
        if proc.poll() is None:
            proc.terminate()
            try:
                proc.wait(timeout=5)
            except Exception:
                proc.kill()
        return None, None

    print(f'Found tunnel URL: {url}')
    print('Waiting for Cloudflare propagation and stable readiness...')

    if wait_reachable_stable(url, proc, warmup_timeout=warmup_timeout, stable_successes=3, probe_interval=2.5):
        print('\n--------------------------------------------------')
        print('YOUR LINK:', url)
        print('--------------------------------------------------\n')
        return proc, url

    print('Tunnel URL did not become stably reachable in time.')
    if proc.poll() is None:
        proc.terminate()
        try:
            proc.wait(timeout=5)
        except Exception:
            proc.kill()
    return None, url


def start_tunnel_with_retries(port=8188, max_attempts=6):
    protocols = ['http2', 'quic']
    for attempt in range(1, max_attempts + 1):
        proto = protocols[(attempt - 1) % len(protocols)]
        print(f"\n== Tunnel attempt {attempt}/{max_attempts} ==")
        proc, url = start_cloudflare_tunnel_once(
            port=port,
            protocol=proto,
            read_timeout=150,
            warmup_timeout=180,
        )
        if proc is not None and url:
            return proc, url
        time.sleep(min(8, 2 + attempt))
    return None, None


def stop_proc_safely(proc):
    if proc is None:
        return
    if proc.poll() is None:
        proc.terminate()
        try:
            proc.wait(timeout=5)
        except Exception:
            proc.kill()


def tunnel_thread(port=8188):
    if not wait_port('127.0.0.1', port, timeout=240):
        print('Timed out waiting for ComfyUI port', port)
        return

    print('\nComfyUI port is open. Creating stable tunnel...\n')

    while True:
        proc, url = start_tunnel_with_retries(port=port, max_attempts=6)
        if proc is None:
            print('Failed to establish a reachable Cloudflare tunnel automatically.')
            print('Rerun this cell to retry with a fresh tunnel session.')
            return

        unhealthy_streak = 0
        while proc.poll() is None:
            time.sleep(15)
            if verify_tunnel_url(url, timeout=8):
                unhealthy_streak = 0
            else:
                unhealthy_streak += 1
                print(f'Cloudflare tunnel health check failed ({unhealthy_streak}/3).')
                if unhealthy_streak >= 3:
                    print('Tunnel became unhealthy. Recreating...')
                    stop_proc_safely(proc)
                    break

        if proc.poll() is not None:
            rc = proc.returncode
            print(f'Cloudflare tunnel exited (code={rc}). Recreating...')


threading.Thread(target=tunnel_thread, daemon=True, args=(8188,)).start()

print('Starting ComfyUI... link will appear above.')
!python main.py --dont-print-server --port 8188 --lowvram --preview-method auto
